In [1]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Pull the known results
df_fixtures = pd.read_csv("https://fixturedownload.com/download/fifa-world-cup-2022-UTC.csv")
df_results = df_fixtures[df_fixtures["Result"].notna()]

In [3]:
# Split Result into home goals and away goals
df_results["Home Goals"] = df_results["Result"].apply(lambda x: int(x.split("-")[0]))
df_results["Away Goals"] = df_results["Result"].apply(lambda x: int(x.split("-")[1]))

In [4]:
# Get the total goals for each team
df_goals = pd.concat(
    [
        df_results[[f"{x} Team", f"{x} Goals"]].rename(columns = lambda x: x.split(" ")[1]) 
        for x in ["Home", "Away"]
    ]
)
df_goals = df_goals.groupby("Team")[["Goals"]].sum().reset_index()

In [5]:
# Read the players picks
df_players_picks = pd.read_csv("df_players_picks.csv")

In [6]:
# Clean the players picks
df_players_picks = df_players_picks.set_index("name")
df_players_picks["picks"] = df_players_picks["picks"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))

In [7]:
# Convert the picks into seperate columns
df_players_goals = pd.DataFrame(df_players_picks["picks"].str.split(", ").tolist(), index=df_players_picks.index)

In [8]:
# Convert to long
df_players_goals = df_players_goals.stack().to_frame("Team")
df_players_goals.index = df_players_goals.index.droplevel(1)

In [9]:
# Join the goals and aggregate to each player
df_players_goals = pd.merge(df_players_goals.reset_index(), df_goals, on="Team", how="left").fillna(0)
df_players_goals = df_players_goals.groupby("name")[["Goals"]].sum().astype(int)

In [10]:
# Join the total goals the the original dataframe
df_players_picks_with_goals = df_players_picks.join(df_players_goals)

In [11]:
df_players_picks_with_goals = df_players_picks_with_goals.sort_values("Goals", ascending=False)

In [12]:
df_players_picks_with_goals

,picks,Goals
name,,
Lee Mayne,"Ecuador, Germany, Netherlands",2
Jack Maddison,"Ecuador, Mexico, Serbia",2
Adam Carrick,"Belgium, Costa Rica, Ecuador",2
Dan Wilson,"Croatia, Ecuador, Germany",2
Adam Ivison,"England, Iran, Uruguay",0
Liam Murtha,"Cameroon, Mexico, Uruguay",0
Claire Hodkinson,"Belgium, England, Ghana",0
Georgia Wilson,"Australia, Belgium, Mexico",0
Danielle Young,"Canada, Senegal, Switzerland",0


In [ ]:
#| echo: true
print(datetime.now())